In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# definition of analytical solution of Displacement BCs
def u_disp(x,z): 
    s13 = -7.22 * 10**(-6)
    sigma_0 = -5
    d15 = 584 * 10**(-9)
    V_0 = 1000
    s33 = 18.8 * 10**(-6)
    L = 1
    u = s13*sigma_0*x
    return u 

def w_disp(x,z): 
    s13 = -7.22 * 10**(-6)
    sigma_0 = -5
    d15 = 584 * 10**(-9)
    V_0 = 1000
    s33 = 18.8 * 10**(-6)
    L = 1
    h = 0.5
    w = (d15*V_0*x)/h + s33*sigma_0*z
    return w 
    
def phi_disp(x,z): 
    s13 = -7.22 * 10**(-6)
    sigma_0 = -5
    d15 = 584 * 10**(-9)
    V_0 = 1000
    s33 = 18.8 * 10**(-6)
    L = 1
    phi = V_0 * (1 - 2 * (x/L))
    return phi
    

In [ ]:
# Meshing for the Piezoelectric Strip under Shear ... 
#  (0,0.5) x ------------x (1,0.5)
#          |             |
#          |             |
#          |             |
#          |             |
# (0,-0.5) x ------------x (1,-0.5)

x = np.array([0,1,1,0])
y = np.array([-0.5, -0.5, 0.5, 0.5])

# Get Max x-length and y-length 
x_length = np.max(x) - np.min(x)
y_length = np.max(y) - np.min(y)

# get mesh points in geometry 
x_linspace = np.linspace(np.min(x), np.max(x), 9, endpoint=True)
y_linspace = np.linspace(np.min(y), np.max(y), 9, endpoint=True)

# loop for making list of points 
x_mesh = []
y_mesh = []
for i in range(len(y_linspace)):
    for j in range(len(x_linspace)):
        x_mesh.append(x_linspace[j])
        y_mesh.append(y_linspace[i])

mesh_file = open('Mesh_data.csv',mode='w')
for i in range(len(x_mesh)):
    mesh_file.write(np.str(i+1))
    mesh_file.write(',')
    mesh_file.write(np.str(x_mesh[i]))
    mesh_file.write(',')
    mesh_file.write(np.str(y_mesh[i]))
    # mesh_file.write(',')
    mesh_file.write('\n')
mesh_file.close()

In [ ]:
# Element Connectivity 
Element_connectivity = open('Ele_conn.csv',mode='w')
# Element_connectivity.write('Element_no, node1, node2, node3, node4')
# Element_connectivity.write('\n')
node_1 = []
node_2 = []
node_3 = []
node_4 = []
for j in range(8): 
      for i in range(8):
        node_1.append((j*(8+1)) + (i+1))
        node_2.append((j*(8+1)) + (i+2))
        node_4.append(((j+1)*(8+1))+(i+1))
        node_3.append(((j+1)*(8+1))+(i+2))
print(np.shape(node_4))        
for i in range(len(node_1)):
    Element_connectivity.write(np.str(i+1))
    Element_connectivity.write(',')
    Element_connectivity.write(np.str(node_1[i]))
    Element_connectivity.write(',')
    Element_connectivity.write(np.str(node_2[i]))
    Element_connectivity.write(',')
    Element_connectivity.write(np.str(node_3[i]))
    Element_connectivity.write(',')
    Element_connectivity.write(np.str(node_4[i]))
    # Element_connectivity.write(',')
    Element_connectivity.write('\n')

Element_connectivity.close()

In [ ]:
# Mesh with 9 x 9 Elements
# Input File Generation for Beam Under Parabolic Loading (Plane Stress)
Input_file = open('Piezoelectric_Strip_Shear.inp', mode='w')

# Heading 
Input_file.write('*HEADING')
Input_file.write('\n')
Input_file.write('Piezoelectric strip under Shear ')
Input_file.write('\n')

# Preprint 
Input_file.write('*PREPRINT, ECHO=YES, HISTORY=NO, MODEL=NO')
Input_file.write('\n')

# Node List 
Input_file.write('*NODE')
Input_file.write('\n')
with open('Mesh_data.csv') as mesh_data:
    for line in mesh_data:
        Input_file.write(line)
# User Element Type 
Input_file.write('*USER ELEMENT, TYPE=U1, NODES=4, PROPERTIES=9,COORDINATES=2, VARIABLES=10')
Input_file.write('\n')
Input_file.write('1, 2, 9')
Input_file.write('\n')

# Element connectivity 
Input_file.write('*ELEMENT, TYPE=U1, ELSET=EALL')
Input_file.write('\n')
with open('Ele_conn.csv') as ele_conn:
      for line in ele_conn:
        Input_file.write(line)

# UEL Property 
Input_file.write('*UEL PROPERTY, ELSET=EALL')
Input_file.write('\n')
Input_file.write('73382.57, 28182.03, 64014.58, 21052.63, -0.02316, -0.0278, 0.012, 1.53105e-8,')
Input_file.write('\n')
Input_file.write('1.505e-8')
Input_file.write('\n')

# Node Sets 
# Boundary 
Input_file.write('*Nset, nset=Set_boundary')
Input_file.write('\n')
Input_file.write('37')
Input_file.write('\n')
# Left Nodes 
Input_file.write('*Nset, nset=left_nodes')
Input_file.write('\n')
Input_file.write('1, 10, 19, 28, 46, 55, 64, 73')
Input_file.write('\n')


# Top Nodes 
Input_file.write('*Nset, nset=Top_nodes')
Input_file.write('\n')
Input_file.write('73, 74, 75, 76, 77, 78, 79, 80, 81')
Input_file.write('\n')



# Bottom Nodes
Input_file.write('*Nset, nset=Bottom_nodes')
Input_file.write('\n')
Input_file.write('1, 2, 3, 4, 5, 6, 7, 8, 9')
Input_file.write('\n')



# Right Nodes
Input_file.write('*Nset, nset=Right_nodes')
Input_file.write('\n')
Input_file.write('9, 18, 27, 36, 45, 54, 63, 72, 81')
Input_file.write('\n')


# Step 
Input_file.write('*STEP')
Input_file.write('\n')
Input_file.write('*STATIC')
Input_file.write('\n')
Input_file.write('1.,1.,1e-05,1.')
Input_file.write('\n')

# Boundary conditions 
Input_file.write('*BOUNDARY')
Input_file.write('\n')
Input_file.write('Set_boundary, 1, 1')
Input_file.write('\n')
Input_file.write('Set_boundary, 2, 2')
Input_file.write('\n')
Input_file.write('Set_boundary, 9, 9, 1000.')
Input_file.write('\n')
Input_file.write('left_nodes, 1, 1')
Input_file.write('\n')
Input_file.write('left_nodes, 9, 9, 1000.')
Input_file.write('\n')
Input_file.write('Right_nodes, 9, 9, -1000.')
Input_file.write('\n')

# Application of Displacement Boundary Conditions ... 
for i in range(len(top_nodes)):
    Input_file.write(np.str(np.int(top_nodes[i]))+', 1, 1,'+np.str(u_1[i]))
    Input_file.write('\n')
    Input_file.write(np.str(np.int(top_nodes[i]))+', 2, 2,'+np.str(v_1[i]))
    Input_file.write('\n')
    Input_file.write(np.str(np.int(top_nodes[i]))+', 9, 9,'+np.str(phi_1[i]))
    Input_file.write('\n')

for i in range(len(bottom_nodes)):
    Input_file.write(np.str(np.int(bottom_nodes[i]))+', 1, 1,'+np.str(u_2[i]))
    Input_file.write('\n')
    Input_file.write(np.str(np.int(bottom_nodes[i]))+', 2, 2,'+np.str(v_2[i]))
    Input_file.write('\n')
    Input_file.write(np.str(np.int(bottom_nodes[i]))+', 9, 9,'+np.str(phi_2[i]))
    Input_file.write('\n')
    
# Output Requests 
Input_file.write('*OUTPUT,FIELD')
Input_file.write('\n')
Input_file.write('*ENERGY FILE')
Input_file.write('\n')
Input_file.write('*OUTPUT,HISTORY,FREQUENCY=1')
Input_file.write('\n')
Input_file.write('*ENERGY OUTPUT,VARIABLE=PRESELECT')
Input_file.write('\n')
Input_file.write('*NODE FILE')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*OUTPUT,FIELD')
Input_file.write('\n')
Input_file.write('*NODE OUTPUT')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*NODE PRINT')
Input_file.write('\n')
Input_file.write('U,COORD')
Input_file.write('\n')
Input_file.write('*EL PRINT')
Input_file.write('\n')
Input_file.write('SDV')
Input_file.write('\n')
Input_file.write('*END STEP')

Input_file.close()